## ROUTE 3 INBOUND 6-9 AM PEAK

In [1]:
import geopandas as gpd 
import requests 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import json
from shapely.geometry import LineString
import folium

%matplotlib inline

In [2]:
with open('../data/WeGo.json')as start:
         credentials = json.load(start)

In [3]:
Authorization = credentials['Authorization']

In [4]:
URL = 'https://api.goswift.ly/speed-map/nashville/route/3'

In [5]:
def concat_frames_columns(geo_response, response, date_range, time_range='6am-9am', days='m-f', in_or_out='inbound'):
    """This function pulls in two API requests, puts one in a geodataframe the other in a dataframe and concats desired columns"""
    geo_df = gpd.GeoDataFrame.from_features(geo_response.json()['features'])
    res_df = pd.DataFrame(response.json()['data']['segments'])
    full_df = pd.concat([geo_df, res_df[['countForSegment', 'percent20Secs']]], axis = 1)
    full_df['dates'] = date_range
    full_df['time'] = time_range
    full_df['days_of_week'] = days
    full_df['direction']= in_or_out
    full_df['max_seg_count'] = full_df['countForSegment'].max()
    full_df['proportion_of_segments'] = full_df['countForSegment']/full_df['max_seg_count']
    return full_df

## In School Months - Jan to May

In [6]:
params1a = {
    'direction':'0',
    'startDate':'01-25-2022',
    'endDate': '02-24-2022',
    'excludeDates': '02-18-2022',
    'beginTime': '04:00',
    'endTime': '06:00',
    'daysOfWeek': '1,2,3,4,5',
    'format':'geojson',
    'resolution':'hiRes',
}


params1b = {
    'direction':'0',
    'startDate':'01-25-2022',
    'endDate': '02-24-2022',
    'excludeDates': '02-18-2022',
    'beginTime': '04:00',
    'endTime': '06:00',
    'daysOfWeek': '1,2,3,4,5',
    #'format':'geojson',
    'resolution':'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [7]:
mfam_jantofeb = concat_frames_columns(requests.get(URL, params = params1a, headers = headers), 
                                      requests.get(URL, params = params1b, headers = headers), "4/27/22-5/25/22")
mfam_jantofeb

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,time,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.78639 36.15708, -86.78663 36.1...",BRO9AWF,Broadway Ave & 9th Ave S Wb,BRO12WN,Broadway Ave & 12th Ave N Wb,24.288,2.601372,9.336612,4.268,3.869,...,1.780,1.712,43,1.836,4/27/22-5/25/22,6am-9am,m-f,inbound,83,0.518072
1,"LINESTRING (-86.81879 36.13973, -86.81897 36.1...",WESACKWN,West End Ave & Acklen Park Dr Wb,WES440WN,West End Ave & I-440 Ramp Wb,24.223,5.025595,4.819927,64.603,9.660,...,1.712,1.396,42,1.852,4/27/22-5/25/22,6am-9am,m-f,inbound,83,0.506024
2,"LINESTRING (-86.82326 36.13553, -86.82348 36.1...",WES440WN,West End Ave & I-440 Ramp Wb,WESBOWWN,West End Ave & Bowling Ave Wb,24.868,1.734143,14.340225,2.450,2.028,...,1.471,1.322,42,1.544,4/27/22-5/25/22,6am-9am,m-f,inbound,83,0.506024
3,"LINESTRING (-86.81680 36.14151, -86.81698 36.1...",WES31AWN,West End Ave & 31st Ave N Wb,WESACKWN,West End Ave & Acklen Park Dr Wb,24.188,3.105512,7.788732,16.055,5.044,...,1.682,1.532,43,1.793,4/27/22-5/25/22,6am-9am,m-f,inbound,83,0.518072
4,"LINESTRING (-86.85538 36.13003, -86.85560 36.1...",WBBTNF,White Bridge Rd & Brookwood Terrace,WHIBRONF,White Bridge Pike & Brookwood Pl Nb,24.347,1.535091,15.860298,1.985,1.776,...,1.254,1.189,22,1.343,4/27/22-5/25/22,6am-9am,m-f,inbound,83,0.265060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,"LINESTRING (-86.85766 36.13797, -86.85761 36.1...",WHIKNONN,White Bridge Pike & Knob Rd Nb,WHIKENNN,White Bridge Pike & Kendall Dr Nb,24.392,2.150000,11.345116,4.072,4.028,...,1.518,1.484,18,1.588,4/27/22-5/25/22,6am-9am,m-f,inbound,83,0.216867
489,"LINESTRING (-86.81570 36.14246, -86.81588 36.1...",WES31AWN,West End Ave & 31st Ave N Wb,WESACKWN,West End Ave & Acklen Park Dr Wb,24.188,2.655977,9.107007,5.312,4.248,...,1.698,1.510,43,1.897,4/27/22-5/25/22,6am-9am,m-f,inbound,83,0.518072
490,"LINESTRING (-86.85427 36.12938, -86.85450 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,1.598455,15.453672,2.122,2.004,...,1.320,1.258,22,1.338,4/27/22-5/25/22,6am-9am,m-f,inbound,83,0.265060
491,"LINESTRING (-86.84662 36.12575, -86.84682 36.1...",HARBOSWM,Harding Pike & Bosley Springs Rd Wb,WHIPOSNN,White Bridge Pike & Post Rd Nb,24.628,2.382652,10.336381,3.565,3.246,...,1.933,1.740,23,2.013,4/27/22-5/25/22,6am-9am,m-f,inbound,83,0.277108


In [8]:
jantofeb = mfam_jantofeb[mfam_jantofeb['percent20Secs'] > np.quantile(mfam_jantofeb['percent20Secs'], .80)]

In [9]:
params2a = {
    'direction':'0',
    'startDate':'02-25-2022',
    'endDate': '03-26-2022',
    'excludeDates': '03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022',
    'beginTime': '04:00',
    'endTime': '06:00',
    'daysOfWeek': '1,2,3,4,5',
    'format':'geojson',
    'resolution':'hiRes',
}


params2b = {
    'direction':'0',
    'startDate':'02-25-2022',
    'endDate': '03-26-2022',
    'excludeDates': '03-14-2022,03-15-2022,03-16-2022,03-17-2022,03-18-2022',
    'beginTime': '04:00',
    'endTime': '06:00',
    'daysOfWeek': '1,2,3,4,5',
    #'format':'geojson',
    'resolution':'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [10]:
mfam_febtomar = concat_frames_columns(requests.get(URL, params = params2a, headers = headers), 
                                      requests.get(URL, params = params2b, headers = headers), "4/27/22-5/25/22")
mfam_febtomar

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,time,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.78639 36.15708, -86.78663 36.1...",BRO9AWF,Broadway Ave & 9th Ave S Wb,BRO12WN,Broadway Ave & 12th Ave N Wb,24.288,2.694188,9.014961,4.532,3.941,...,1.949,1.883,32,2.032,4/27/22-5/25/22,6am-9am,m-f,inbound,45,0.711111
1,"LINESTRING (-86.81879 36.13973, -86.81897 36.1...",WESACKWN,West End Ave & Acklen Park Dr Wb,WES440WN,West End Ave & I-440 Ramp Wb,24.223,6.134000,3.948973,64.668,12.232,...,1.727,1.604,32,1.853,4/27/22-5/25/22,6am-9am,m-f,inbound,45,0.711111
2,"LINESTRING (-86.82326 36.13553, -86.82348 36.1...",WES440WN,West End Ave & I-440 Ramp Wb,WESBOWWN,West End Ave & Bowling Ave Wb,24.868,1.876469,13.252550,2.631,2.260,...,1.553,1.409,32,1.623,4/27/22-5/25/22,6am-9am,m-f,inbound,45,0.711111
3,"LINESTRING (-86.81657 36.14171, -86.81675 36.1...",WES31AWN,West End Ave & 31st Ave N Wb,WESACKWN,West End Ave & Acklen Park Dr Wb,24.188,2.638409,9.167646,4.752,4.039,...,1.758,1.653,22,1.855,4/27/22-5/25/22,6am-9am,m-f,inbound,45,0.488889
4,"LINESTRING (-86.85538 36.13003, -86.85560 36.1...",WBBTNF,White Bridge Rd & Brookwood Terrace,WHIBRONF,White Bridge Pike & Brookwood Pl Nb,24.347,1.634063,14.899670,2.276,1.983,...,1.449,1.410,16,1.499,4/27/22-5/25/22,6am-9am,m-f,inbound,45,0.355556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,"LINESTRING (-86.81377 36.14412, -86.81396 36.1...",WES27AWN,West End Ave & 27th Ave S Wb,WES31AWN,West End Ave & 31st Ave N Wb,24.810,2.305100,10.763091,2.887,2.886,...,1.808,1.803,10,1.914,4/27/22-5/25/22,6am-9am,m-f,inbound,45,0.222222
519,"LINESTRING (-86.81547 36.14266, -86.81565 36.1...",WES31AWN,West End Ave & 31st Ave N Wb,WESACKWN,West End Ave & Acklen Park Dr Wb,24.188,2.809000,8.610894,4.758,3.737,...,1.988,1.953,22,2.109,4/27/22-5/25/22,6am-9am,m-f,inbound,45,0.488889
520,"LINESTRING (-86.85427 36.12938, -86.85450 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,1.687625,14.637138,2.341,2.021,...,1.509,1.487,16,1.545,4/27/22-5/25/22,6am-9am,m-f,inbound,45,0.355556
521,"LINESTRING (-86.84662 36.12575, -86.84682 36.1...",HARBOSWM,Harding Pike & Bosley Springs Rd Wb,WHIPOSNN,White Bridge Pike & Post Rd Nb,24.628,2.449750,10.053271,2.926,2.804,...,1.995,1.969,16,2.218,4/27/22-5/25/22,6am-9am,m-f,inbound,45,0.355556


In [11]:
febtomar = mfam_febtomar[mfam_febtomar['percent20Secs'] > np.quantile(mfam_febtomar['percent20Secs'], .80)]

In [12]:
params3a = {
    'direction':'0',
    'startDate':'03-27-2022',
    'endDate': '04-26-2022',
    'excludeDates': '04-14-2022,04-15-2022',
    'beginTime': '04:00',
    'endTime': '06:00',
    'daysOfWeek': '1,2,3,4,5',
    'format':'geojson',
    'resolution':'hiRes',
}


params3b = {
    'direction':'0',
    'startDate':'03-27-2022',
    'endDate': '04-26-2022',
    'excludeDates': '04-14-2022,04-15-2022',
    'beginTime': '04:00',
    'endTime': '06:00',
    'daysOfWeek': '1,2,3,4,5',
    #'format':'geojson',
    'resolution':'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [13]:
mfam_martoapr = concat_frames_columns(requests.get(URL, params = params3a, headers = headers), 
                                      requests.get(URL, params = params3b, headers = headers), "4/27/22-5/25/22")
mfam_martoapr

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,time,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.78639 36.15708, -86.78663 36.1...",BRO9AWF,Broadway Ave & 9th Ave S Wb,BRO12WN,Broadway Ave & 12th Ave N Wb,24.288,2.547744,9.533140,4.241,3.797,...,1.751,1.435,39,1.972,4/27/22-5/25/22,6am-9am,m-f,inbound,41,0.951220
1,"LINESTRING (-86.81879 36.13973, -86.81897 36.1...",WESACKWN,West End Ave & Acklen Park Dr Wb,WES440WN,West End Ave & I-440 Ramp Wb,24.223,6.100975,3.970349,37.312,20.006,...,1.836,1.695,40,1.986,4/27/22-5/25/22,6am-9am,m-f,inbound,41,0.975610
2,"LINESTRING (-86.82326 36.13553, -86.82348 36.1...",WES440WN,West End Ave & I-440 Ramp Wb,WESBOWWN,West End Ave & Bowling Ave Wb,24.868,1.879923,13.228201,2.418,2.281,...,1.605,1.538,39,1.667,4/27/22-5/25/22,6am-9am,m-f,inbound,41,0.951220
3,"LINESTRING (-86.85538 36.13003, -86.85560 36.1...",WBBTNF,White Bridge Rd & Brookwood Terrace,WHIBRONF,White Bridge Pike & Brookwood Pl Nb,24.347,1.552750,15.679923,1.904,1.820,...,1.341,1.304,20,1.369,4/27/22-5/25/22,6am-9am,m-f,inbound,41,0.487805
4,"LINESTRING (-86.85360 36.12899, -86.85383 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,1.659900,14.881619,2.031,1.838,...,1.434,1.423,20,1.518,4/27/22-5/25/22,6am-9am,m-f,inbound,41,0.487805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,"LINESTRING (-86.85766 36.13797, -86.85761 36.1...",WHIKNONN,White Bridge Pike & Knob Rd Nb,WHIKENNN,White Bridge Pike & Kendall Dr Nb,24.392,2.253571,10.823710,4.530,4.198,...,1.340,1.318,14,1.387,4/27/22-5/25/22,6am-9am,m-f,inbound,41,0.341463
488,"LINESTRING (-86.81377 36.14412, -86.81396 36.1...",WES27AWN,West End Ave & 27th Ave S Wb,WES31AWN,West End Ave & 31st Ave N Wb,24.810,2.207175,11.240613,3.621,2.489,...,1.877,1.763,40,1.978,4/27/22-5/25/22,6am-9am,m-f,inbound,41,0.975610
489,"LINESTRING (-86.85427 36.12938, -86.85450 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,1.644800,15.018239,1.870,1.838,...,1.434,1.423,20,1.518,4/27/22-5/25/22,6am-9am,m-f,inbound,41,0.487805
490,"LINESTRING (-86.84662 36.12575, -86.84682 36.1...",HARBOSWM,Harding Pike & Bosley Springs Rd Wb,WHIPOSNN,White Bridge Pike & Post Rd Nb,24.628,2.524560,9.755363,5.323,3.467,...,1.867,1.755,25,2.020,4/27/22-5/25/22,6am-9am,m-f,inbound,41,0.609756


In [14]:
martoapr = mfam_martoapr[mfam_martoapr['percent20Secs'] > np.quantile(mfam_martoapr['percent20Secs'], .80)]

In [15]:
params4a = {
    'direction': '0',
    'startDate': '04-27-2022',
    'beginTime' : '04:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '06:00',
    'format' : 'geojson',
    'resolution': 'hiRes',
}


params4b = {
    'direction': '0',
    'startDate': '04-27-2022',
    'beginTime' : '04:00',
    'daysOfWeek' : '1,2,3,4,5',
    'endDate' : '05-25-2022',
    'endTime' : '06:00',
    'resolution': 'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [16]:
mfam_aprtomay = concat_frames_columns(requests.get(URL, params = params4a, headers = headers), 
                                      requests.get(URL, params = params4b, headers = headers), "4/27/22-5/25/22")
mfam_aprtomay

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,time,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.78639 36.15708, -86.78663 36.1...",BRO9AWF,Broadway Ave & 9th Ave S Wb,BRO12WN,Broadway Ave & 12th Ave N Wb,24.288,2.789342,8.707430,4.834,3.983,...,1.807,1.416,38,2.054,4/27/22-5/25/22,6am-9am,m-f,inbound,39,0.974359
1,"LINESTRING (-86.81879 36.13973, -86.81897 36.1...",WESACKWN,West End Ave & Acklen Park Dr Wb,WES440WN,West End Ave & I-440 Ramp Wb,24.223,3.161282,7.662398,12.559,6.415,...,1.737,1.497,39,1.770,4/27/22-5/25/22,6am-9am,m-f,inbound,39,1.000000
2,"LINESTRING (-86.82326 36.13553, -86.82348 36.1...",WES440WN,West End Ave & I-440 Ramp Wb,WESBOWWN,West End Ave & Bowling Ave Wb,24.868,1.860158,13.368757,3.278,2.282,...,1.411,1.291,38,1.590,4/27/22-5/25/22,6am-9am,m-f,inbound,39,0.974359
3,"LINESTRING (-86.85538 36.13003, -86.85560 36.1...",WBBTNF,White Bridge Rd & Brookwood Terrace,WHIBRONF,White Bridge Pike & Brookwood Pl Nb,24.347,1.566421,15.543076,1.921,1.874,...,1.412,1.266,19,1.473,4/27/22-5/25/22,6am-9am,m-f,inbound,39,0.487179
4,"LINESTRING (-86.85360 36.12899, -86.85383 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,1.838571,13.435434,4.899,2.626,...,1.480,1.284,21,1.523,4/27/22-5/25/22,6am-9am,m-f,inbound,39,0.538462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,"LINESTRING (-86.85766 36.13797, -86.85761 36.1...",WHIKNONN,White Bridge Pike & Knob Rd Nb,WHIKENNN,White Bridge Pike & Kendall Dr Nb,24.392,2.255200,10.815892,5.347,4.515,...,1.440,1.412,15,1.538,4/27/22-5/25/22,6am-9am,m-f,inbound,39,0.384615
487,"LINESTRING (-86.81377 36.14412, -86.81396 36.1...",WES27AWN,West End Ave & 27th Ave S Wb,WES31AWN,West End Ave & 31st Ave N Wb,24.810,2.299789,10.787946,7.119,2.646,...,1.581,1.377,38,1.759,4/27/22-5/25/22,6am-9am,m-f,inbound,39,0.974359
488,"LINESTRING (-86.85427 36.12938, -86.85450 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,1.598895,15.449420,2.341,1.696,...,1.478,1.284,19,1.495,4/27/22-5/25/22,6am-9am,m-f,inbound,39,0.487179
489,"LINESTRING (-86.84662 36.12575, -86.84682 36.1...",HARBOSWM,Harding Pike & Bosley Springs Rd Wb,WHIPOSNN,White Bridge Pike & Post Rd Nb,24.628,2.309444,10.664039,2.958,2.868,...,1.813,1.646,27,1.989,4/27/22-5/25/22,6am-9am,m-f,inbound,39,0.692308


In [17]:
aprtomay = mfam_aprtomay[mfam_aprtomay['percent20Secs'] > np.quantile(mfam_aprtomay['percent20Secs'], .80)]

## Summer Months

In [18]:
params5a = {
    'direction':'0',
    'startDate':'05-26-2022',
    'endDate': '06-25-2022',
    'beginTime': '04:00',
    'endTime': '06:00',
    'daysOfWeek': '1,2,3,4,5',
    'format':'geojson',
    'resolution':'hiRes',
}


params5b = {
    'direction':'0',
    'startDate':'05-26-2022',
    'endDate': '06-25-2022',
    'beginTime': '04:00',
    'endTime': '06:00',
    'daysOfWeek': '1,2,3,4,5',
    #'format':'geojson',
    'resolution':'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [19]:
mfam_maytojun = concat_frames_columns(requests.get(URL, params = params5a, headers = headers), 
                                      requests.get(URL, params = params5b, headers = headers), "4/27/22-5/25/22")
mfam_maytojun

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,time,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.78639 36.15708, -86.78663 36.1...",BRO9AWF,Broadway Ave & 9th Ave S Wb,BRO12WN,Broadway Ave & 12th Ave N Wb,24.288,2.562561,9.478018,4.054,3.662,...,1.786,1.684,41,1.932,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.976190
1,"LINESTRING (-86.81879 36.13973, -86.81897 36.1...",WESACKWN,West End Ave & Acklen Park Dr Wb,WES440WN,West End Ave & I-440 Ramp Wb,24.223,9.222925,2.626390,46.936,31.075,...,1.641,1.335,40,1.763,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.952381
2,"LINESTRING (-86.82326 36.13553, -86.82348 36.1...",WES440WN,West End Ave & I-440 Ramp Wb,WESBOWWN,West End Ave & Bowling Ave Wb,24.868,1.822100,13.647989,2.685,2.123,...,1.465,1.233,40,1.594,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.952381
3,"LINESTRING (-86.85538 36.13003, -86.85560 36.1...",WBBTNF,White Bridge Rd & Brookwood Terrace,WHIBRONF,White Bridge Pike & Brookwood Pl Nb,24.347,1.578952,15.419721,2.384,2.237,...,1.345,1.175,21,1.418,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.500000
4,"LINESTRING (-86.85360 36.12899, -86.85383 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,1.714190,14.410305,3.009,2.353,...,1.327,1.019,21,1.476,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,"LINESTRING (-86.85766 36.13797, -86.85761 36.1...",WHIKNONN,White Bridge Pike & Knob Rd Nb,WHIKENNN,White Bridge Pike & Kendall Dr Nb,24.392,2.494471,9.778426,6.440,4.354,...,1.351,1.339,17,1.455,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.404762
488,"LINESTRING (-86.81377 36.14412, -86.81396 36.1...",WES27AWN,West End Ave & 27th Ave S Wb,WES31AWN,West End Ave & 31st Ave N Wb,24.810,2.230190,11.124613,8.048,2.584,...,1.668,1.261,42,1.827,4/27/22-5/25/22,6am-9am,m-f,inbound,42,1.000000
489,"LINESTRING (-86.85427 36.12938, -86.85450 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,1.676714,14.732387,4.566,2.271,...,1.324,1.019,21,1.396,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.500000
490,"LINESTRING (-86.84662 36.12575, -86.84682 36.1...",HARBOSWM,Harding Pike & Bosley Springs Rd Wb,WHIPOSNN,White Bridge Pike & Post Rd Nb,24.628,2.245633,10.967064,3.077,2.861,...,1.690,1.615,30,1.851,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.714286


In [20]:
maytojun = mfam_maytojun[mfam_maytojun['percent20Secs'] > np.quantile(mfam_maytojun['percent20Secs'], .80)]

In [21]:
params6a = {
    'direction':'0',
    'startDate':'06-26-2022',
    'endDate': '07-26-2022',
    'excludeDate': '07/04/2022',
    'beginTime': '04:00',
    'endTime': '06:00',
    'daysOfWeek': '1,2,3,4,5',
    'format':'geojson',
    'resolution':'hiRes',
}


params6b = {
    'direction':'0',
    'startDate':'06-26-2022',
    'endDate': '07-26-2022',
    'excludeDate': '07/04/2022',
    'beginTime': '04:00',
    'endTime': '06:00',
    'daysOfWeek': '1,2,3,4,5',
    #'format':'geojson',
    'resolution':'hiRes',
}

headers = {'Authorization': Authorization,
          'Content-Type': 'application/json'}

In [22]:
mfam_juntojul = concat_frames_columns(requests.get(URL, params = params6a, headers = headers), 
                                      requests.get(URL, params = params6b, headers = headers), "4/27/22-5/25/22")
mfam_juntojul

,geometry,fromStopId,fromStopName,toStopId,toStopName,pathLengthMeters,averageTravelSeconds,averageTravelSpeed,maxSecs,percent90Secs,...,percent10Secs,minSecs,countForSegment,percent20Secs,dates,time,days_of_week,direction,max_seg_count,proportion_of_segments
0,"LINESTRING (-86.78639 36.15708, -86.78663 36.1...",BRO9AWF,Broadway Ave & 9th Ave S Wb,BRO12WN,Broadway Ave & 12th Ave N Wb,24.288,2.648450,9.170647,6.215,3.713,...,1.787,1.460,40,2.012,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.952381
1,"LINESTRING (-86.81879 36.13973, -86.81897 36.1...",WESACKWN,West End Ave & Acklen Park Dr Wb,WES440WN,West End Ave & I-440 Ramp Wb,24.223,3.534300,6.853691,15.077,7.971,...,1.612,1.336,40,1.709,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.952381
2,"LINESTRING (-86.82326 36.13553, -86.82348 36.1...",WES440WN,West End Ave & I-440 Ramp Wb,WESBOWWN,West End Ave & Bowling Ave Wb,24.868,1.679825,14.803923,2.078,2.006,...,1.445,1.312,40,1.491,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.952381
3,"LINESTRING (-86.85538 36.13003, -86.85560 36.1...",WBBTNF,White Bridge Rd & Brookwood Terrace,WHIBRONF,White Bridge Pike & Brookwood Pl Nb,24.347,1.594400,15.270321,2.210,1.890,...,1.422,1.355,20,1.457,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.476190
4,"LINESTRING (-86.85360 36.12899, -86.85383 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,1.706500,14.475242,2.395,2.212,...,1.446,1.340,20,1.498,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.476190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,"LINESTRING (-86.85766 36.13797, -86.85761 36.1...",WHIKNONN,White Bridge Pike & Knob Rd Nb,WHIKENNN,White Bridge Pike & Kendall Dr Nb,24.392,2.886500,8.450372,5.750,5.561,...,1.332,1.291,12,1.444,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.285714
488,"LINESTRING (-86.81377 36.14412, -86.81396 36.1...",WES27AWN,West End Ave & 27th Ave S Wb,WES31AWN,West End Ave & 31st Ave N Wb,24.810,2.008175,12.354501,3.266,2.761,...,1.417,1.325,40,1.677,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.952381
489,"LINESTRING (-86.85427 36.12938, -86.85450 36.1...",WHIPOSNN,White Bridge Pike & Post Rd Nb,WBBTNF,White Bridge Rd & Brookwood Terrace,24.702,1.650250,14.968641,2.395,2.216,...,1.382,1.340,20,1.482,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.476190
490,"LINESTRING (-86.84662 36.12575, -86.84682 36.1...",HARBOSWM,Harding Pike & Bosley Springs Rd Wb,WHIPOSNN,White Bridge Pike & Post Rd Nb,24.628,2.284156,10.782101,3.089,2.925,...,1.664,1.505,32,1.826,4/27/22-5/25/22,6am-9am,m-f,inbound,42,0.761905


In [23]:
juntojul = mfam_juntojul[mfam_juntojul['percent20Secs'] > np.quantile(mfam_juntojul['percent20Secs'], .80)]

In [24]:
school = pd.concat([jantofeb, febtomar, martoapr, aprtomay], keys=['jantofeb','febtomar','martoapr','aprtomay'])
summer = pd.concat([maytojun, juntojul], keys = ['maytojun', 'juntojul'])
r3oomschool = school['geometry'].to_json()
r3oomsummer = summer['geometry'].to_json()
mapdawg = folium.Map(location = [36.19, -86.7775], zoom_start = 12)

In [25]:
style1 = {'color' : 'red', 'weight' : 6}
style2 = {'color' : 'blue', 'weight' : 3}
folium.GeoJson(r3oomschool, style_function =lambda x:style1).add_to(mapdawg)
folium.GeoJson(r3oomsummer, style_function = lambda x:style2).add_to(mapdawg)
mapdawg